In [13]:
## ensemble y entrega final

In [14]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-03 18:49:52 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,774459,41.4,1458797,78.0,1458797,78.0
Vcells,1568214,12.0,1422339080,10851.6,1449739584,11060.7


In [15]:
setwd("~")

In [16]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset1 <- fread("~/datasets/competencia_02_crudo.csv.gz" )
dataset2<- fread("~/datasets/competencia_03_crudo.csv.gz" )

dataset <- rbind(dataset1, dataset2)


# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-12-03 18:49:53 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,775385,41.5,1458797,78.0,1458797,78.0
Vcells,1301399643,9928.9,1706886896,13022.6,1449739584,11060.7


[1] "2025-12-03 18:50:21 UTC"

In [17]:
dfuture <- dataset[foto_mes == 202109]

In [18]:
# 2) Traigo archivos de predicciones
files <- list.files("/home/guadalesandro/buckets/b1/exp/C3",
                    recursive=TRUE,
                    full.names=TRUE)

print(files)

 [1] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_b2.txt"
 [2] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_d.txt" 
 [3] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_e.txt" 
 [4] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_e2.txt"
 [5] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_f.txt" 
 [6] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_h.txt" 
 [7] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_j.txt" 
 [8] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_k.txt" 
 [9] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_k2.txt"
[10] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_k3.txt"
[11] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_k4.txt"
[12] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_k5.txt"
[13] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_kk.txt"
[14] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_n5.txt"
[15] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_n6.txt"
[16] "/home/guadalesandro/buckets/b1/exp/C3/prediccion_

In [20]:
# 1 - Levanto predicciones y concateno
pred_list <- lapply(files, function(f){
  fread(f, sep="\t", header=TRUE, encoding="UTF-8")})

pred <- rbindlist(pred_list, use.names=TRUE, fill=TRUE)


pred[, numero_de_cliente := as.character(numero_de_cliente)]
pred[, prob := as.numeric(prob)]


# 2 - promedia probabilidades y las ordena
pred_ensamble <- pred[, .(prob_mean = mean(prob, na.rm=TRUE)), by = numero_de_cliente]
setorder(pred_ensamble, -prob_mean)


# 3 - Filtro solo clientes de dfuture
clientes_future <- unique(dfuture$numero_de_cliente)
pred_ensamble <- pred_ensamble[numero_de_cliente %in% clientes_future]

#4 - Cantidad de envíos
envios <- 11000
pred_final <- pred_ensamble[1:envios]


cat("Cantidad de registros en el archivo final:", nrow(pred_final), "\n")


# 5 - Guardo el archivo en carpeta final
output_dir <- "/home/guadalesandro/buckets/b1/exp/C3/"

if(!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)
archivo_final <- file.path(output_dir, "C3_final_Alesandro.csv")

fwrite(
  pred_final[, .(numero_de_cliente)],
  file = archivo_final,
  sep = ",",
  col.names = FALSE
)

cat("Archivo guardado en:", archivo_final, "\n")

Cantidad de registros en el archivo final: 11000 
Archivo guardado en: /home/guadalesandro/buckets/b1/exp/C3//C3_final_Alesandro.csv 
